# 数据处理

## 数据导入

In [21]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# 加载原始数据集

# 资产价格数据 (Assets)
df_gold = pd.read_csv('../data/raw/Gold.csv', parse_dates=['Date'])
df_wti = pd.read_csv('../data/raw/WTI.csv', parse_dates=['Date'])
df_silver = pd.read_csv('../data/raw/Silver.csv', parse_dates=['Date'])
df_djia = pd.read_csv('../data/raw/DJIA.csv', parse_dates=['Date'])
df_usdx = pd.read_csv('../data/raw/USDX.csv', parse_dates=['Date'])

# 利率数据 (Rates)
df_us10yt = pd.read_csv('../data/raw/US10YT.csv', parse_dates=['Date'])
df_effr = pd.read_csv('../data/raw/EFFR.csv', parse_dates=['Date'])
df_t10yie = pd.read_csv('../data/raw/T10YIE.csv', parse_dates=['Date'])

# 指数数据 (Indices)
df_vix = pd.read_csv('../data/raw/VIX.csv', parse_dates=['Date'])
df_cpi = pd.read_csv('../data/raw/CPI.csv', parse_dates=['Date'])

# 风险数据 (Risk)
df_gpr = pd.read_csv('../data/raw/GPR.csv', parse_dates=['Date'])

print(f"Date time range from: {df_gold['Date'].min().date()} to {df_gold['Date'].max().date()}")

Date time range from: 2005-01-03 to 2025-12-30


In [22]:
# 设置以时间索引
def format_timeseries(df, col_name):
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.drop_duplicates(subset=['Date']).set_index('Date').sort_index()
    df.columns = [col_name]
    return df

df_gold = format_timeseries(df_gold, 'Gold')
df_wti = format_timeseries(df_wti, 'WTI')
df_silver = format_timeseries(df_silver, 'Silver')
df_djia = format_timeseries(df_djia, 'DJIA')
df_usdx = format_timeseries(df_usdx, 'USDX')
df_us10yt = format_timeseries(df_us10yt, 'US10YT')
df_effr = format_timeseries(df_effr, 'EFFR')
df_t10yie = format_timeseries(df_t10yie, 'T10YIE')
df_vix = format_timeseries(df_vix, 'VIX')
df_cpi = format_timeseries(df_cpi, 'CPI')
df_gpr = format_timeseries(df_gpr, 'GPR')

## 内生变量

In [23]:
# 计算RSI (14日)
delta = df_gold['Gold'].diff()
gain = delta.where(delta > 0, 0).ewm(alpha=1/14, adjust=False).mean()
loss = (-delta.where(delta < 0, 0)).ewm(alpha=1/14, adjust=False).mean()
df_gold['RSI'] = 100 - (100 / (1 + gain / loss))

# 计算布林带宽度（Bollinger Band Width） (20日, 2倍标准差)
std20 = df_gold['Gold'].rolling(window=20).std()
sma20 = df_gold['Gold'].rolling(window=20).mean() # 计算中轨
df_gold['BBW'] = (4 * std20) / sma20 # 标准相对宽度公式

# 计算MACD
ema12 = df_gold['Gold'].ewm(span=12, adjust=False).mean()
ema26 = df_gold['Gold'].ewm(span=26, adjust=False).mean()
df_gold['MACD'] = ema12 - ema26

In [24]:
df_gold

,Gold,RSI,BBW,MACD
Date,,,,
2005-01-03,429.7,NaN,NaN,0.000000
2005-01-04,429.2,0.000000,NaN,-0.039886
2005-01-05,427.3,0.000000,NaN,-0.222248
2005-01-06,421.6,0.000000,NaN,-0.817293
2005-01-07,419.5,0.000000,NaN,-1.441703
...,...,...,...,...
2025-12-22,4469.4,73.533683,0.098508,87.982430
2025-12-23,4505.7,75.221799,0.108884,96.898848
2025-12-25,4527.5,76.203450,0.116570,104.519417


In [25]:
# 计算金价对数收益率
df_gold['Gold_log'] = np.log(df_gold['Gold'] / df_gold['Gold'].shift(1))

In [26]:
df_gold

,Gold,RSI,BBW,MACD,Gold_log
Date,,,,,
2005-01-03,429.7,NaN,NaN,0.000000,NaN
2005-01-04,429.2,0.000000,NaN,-0.039886,-0.001164
2005-01-05,427.3,0.000000,NaN,-0.222248,-0.004437
2005-01-06,421.6,0.000000,NaN,-0.817293,-0.013429
2005-01-07,419.5,0.000000,NaN,-1.441703,-0.004993
...,...,...,...,...,...
2025-12-22,4469.4,73.533683,0.098508,87.982430,0.023751
2025-12-23,4505.7,75.221799,0.108884,96.898848,0.008089
2025-12-25,4527.5,76.203450,0.116570,104.519417,0.004827


## 平稳化转换（Point-in-Time Protocol）

In [27]:
# 1. 资产价格类：对数收益率转化
for df, col in zip([df_wti, df_silver, df_djia, df_usdx], ['WTI', 'Silver', 'DJIA', 'USDX']):
    df[col] = np.log(df[col]).diff()

# 2. 利率与风险类：一阶差分处理
for df, col in zip([df_us10yt, df_effr, df_t10yie, df_vix], ['US10YT', 'EFFR', 'T10YIE', 'VIX']):
    df[col] = df[col].diff()

# 3. 地缘风险：对数化处理
df_gpr['GPR'] = np.log(df_gpr['GPR'])

## PiT修正 (Point-in-Time Protocol)

In [28]:
# 4. 宏观指标：强制执行45日时滞 (PiT)
df_cpi = df_cpi.reset_index()
df_cpi['Date_Available'] = df_cpi['Date'] + pd.Timedelta(days=45)
df_cpi_pit = df_cpi[['Date_Available', 'CPI']].set_index('Date_Available')
df_cpi = df_cpi_pit.resample('D').ffill()
df_cpi.index.name = 'Date'

In [29]:
df_cpi

,CPI
Date,
2005-02-15,199.000
2005-02-16,199.000
2005-02-17,199.000
2005-02-18,199.000
2005-02-19,199.000
...,...
2026-01-11,331.068
2026-01-12,331.068
2026-01-13,331.068


In [30]:
# 合并数据集
dfs_to_merge = [df_wti, df_silver, df_djia, df_usdx, df_us10yt, df_effr, df_t10yie, df_vix, df_gpr, df_cpi]
df_merged = df_gold.join(dfs_to_merge, how='left').ffill()

# 按指定顺序重排特征列
target_cols = ['Gold','Gold_log', 'WTI', 'Silver', 'DJIA', 'USDX', 'US10YT', 'EFFR', 'T10YIE', 'VIX', 'GPR', 'CPI', 'RSI', 'BBW', 'MACD']
df_merged = df_merged[target_cols]

In [31]:
df_merged

,Gold,Gold_log,WTI,Silver,DJIA,USDX,US10YT,EFFR,T10YIE,VIX,GPR,CPI,RSI,BBW,MACD
Date,,,,,,,,,,,,,,,
2005-01-03,429.7,NaN,NaN,NaN,-0.004981,NaN,NaN,NaN,NaN,NaN,4.955123,NaN,NaN,NaN,0.000000
2005-01-04,429.2,-0.001164,0.041808,-0.008488,-0.009237,0.015500,0.079,-0.06,-0.02,-0.10,4.980657,NaN,0.000000,NaN,-0.039886
2005-01-05,427.3,-0.004437,-0.012590,0.012782,-0.003104,-0.000363,-0.008,0.00,-0.01,0.11,4.699571,NaN,0.000000,NaN,-0.222248
2005-01-06,421.6,-0.013429,0.047242,-0.012317,0.002361,0.007363,-0.024,0.00,0.03,-0.51,4.489310,NaN,0.000000,NaN,-0.817293
2005-01-07,419.5,-0.004993,-0.004184,-0.001085,-0.001783,0.005517,0.012,-0.01,-0.04,-0.09,4.682594,NaN,0.000000,NaN,-1.441703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-22,4469.4,0.023751,0.024005,0.036825,0.004721,-0.001449,0.020,0.00,-0.01,-0.83,4.613337,331.068,73.533683,0.098508,87.982430
2025-12-23,4505.7,0.008089,0.006339,0.007674,0.001647,-0.002382,-0.002,0.00,0.01,-0.08,4.600359,331.068,75.221799,0.108884,96.898848
2025-12-25,4527.5,0.004827,0.006339,0.074066,0.001647,-0.002382,-0.001,0.00,0.01,-0.08,4.991996,331.068,76.203450,0.116570,104.519417


In [32]:
# 最终调整：删除原始金价列，重命名对数收益率列
df_merged.drop(columns=['Gold'], inplace=True)
df_merged.rename(columns={'Gold_log': 'Gold'}, inplace=True)
df_merged.to_csv('../data/processed/consolidated_features.csv', index=True)

In [33]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4959 entries, 2005-01-03 to 2025-12-30
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Gold    4958 non-null   float64
 1   WTI     4958 non-null   float64
 2   Silver  4958 non-null   float64
 3   DJIA    4959 non-null   float64
 4   USDX    4958 non-null   float64
 5   US10YT  4958 non-null   float64
 6   EFFR    4958 non-null   float64
 7   T10YIE  4958 non-null   float64
 8   VIX     4958 non-null   float64
 9   GPR     4959 non-null   float64
 10  CPI     4929 non-null   float64
 11  RSI     4958 non-null   float64
 12  BBW     4940 non-null   float64
 13  MACD    4959 non-null   float64
dtypes: float64(14)
memory usage: 710.2 KB


## 构造前瞻性标签

In [34]:
# 构造前瞻性标签：t+1时刻的黄金对数收益率
df_merged['Target'] = df_merged['Gold'].shift(-1)

# 截取 2006.01.01-2025.10.31 的样本，并剔除由于shift产生的NaN
df_sample = df_merged.loc['2006-01-01':'2025-10-31'].dropna(subset=['Target'])

# 按照 7:1:2 进行非随机时序划分
n_samples = len(df_sample)
train_idx = int(n_samples * 0.7)
val_idx = train_idx + int(n_samples * 0.1)

train_set = df_sample.iloc[:train_idx]
val_set = df_sample.iloc[train_idx:val_idx]
test_set = df_sample.iloc[val_idx:]

# 分离特征 (X) 与标签 (y)
X_train, y_train = train_set.drop(columns=['Target']), train_set['Target']
X_val, y_val = val_set.drop(columns=['Target']), val_set['Target']
X_test, y_test = test_set.drop(columns=['Target']), test_set['Target']

In [35]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4676 entries, 2006-01-03 to 2025-10-30
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Gold    4676 non-null   float64
 1   WTI     4676 non-null   float64
 2   Silver  4676 non-null   float64
 3   DJIA    4676 non-null   float64
 4   USDX    4676 non-null   float64
 5   US10YT  4676 non-null   float64
 6   EFFR    4676 non-null   float64
 7   T10YIE  4676 non-null   float64
 8   VIX     4676 non-null   float64
 9   GPR     4676 non-null   float64
 10  CPI     4676 non-null   float64
 11  RSI     4676 non-null   float64
 12  BBW     4676 non-null   float64
 13  MACD    4676 non-null   float64
 14  Target  4676 non-null   float64
dtypes: float64(15)
memory usage: 584.5 KB


## 数据划分

In [36]:
from sklearn.preprocessing import StandardScaler

# 初始化标准化器
scaler = StandardScaler()

# 仅基于训练集计算均值与标准差，并转换训练集
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)

# 将训练集统计量同步应用于验证集与测试集
X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns, index=X_val.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

In [37]:
# 构造不同窗口大小的序列数据
def create_sequences(X, y, window_size):
    X_vals, y_vals = X.values, y.values
    Xs, ys = [], []
    for i in range(len(X) - window_size + 1):
        Xs.append(X_vals[i : i + window_size])
        ys.append(y_vals[i + window_size - 1])
    return np.array(Xs), np.array(ys)

windows = [5, 21, 63]
experiment_data = {}

for w in windows:
    experiment_data[w] = {
        'X_train': create_sequences(X_train_scaled, y_train, w)[0],
        'y_train': create_sequences(X_train_scaled, y_train, w)[1],
        'X_val': create_sequences(X_val_scaled, y_val, w)[0],
        'y_val': create_sequences(X_val_scaled, y_val, w)[1],
        'X_test': create_sequences(X_test_scaled, y_test, w)[0],
        'y_test': create_sequences(X_test_scaled, y_test, w)[1]
    }

In [38]:
# 输出每个窗口大小对应的数据形状
for w, data in experiment_data.items():
    print(f"Window Size: {w}")
    print(f"  X_train shape: {data['X_train'].shape}")
    print(f"  X_val shape:   {data['X_val'].shape}")
    print(f"  X_test shape:  {data['X_test'].shape}")
    print("-" * 30)

Window Size: 5
  X_train shape: (3269, 5, 14)
  X_val shape:   (463, 5, 14)
  X_test shape:  (932, 5, 14)
------------------------------
Window Size: 21
  X_train shape: (3253, 21, 14)
  X_val shape:   (447, 21, 14)
  X_test shape:  (916, 21, 14)
------------------------------
Window Size: 63
  X_train shape: (3211, 63, 14)
  X_val shape:   (405, 63, 14)
  X_test shape:  (874, 63, 14)
------------------------------


# 特征筛选

In [39]:
# from sklearn.ensemble import RandomForestRegressor
# import shap
# import pandas as pd

# # 针对当前 2D 特征（未进行3D滑动窗口前）训练随机森林，计算全局 G-SHAP 与相对重要性
# rf_shap = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
# rf_shap.fit(X_train_scaled, y_train)

# # 计算 SHAP 值
# explainer = shap.TreeExplainer(rf_shap)
# shap_values = explainer.shap_values(X_train_scaled)

# # 计算 G-SHAP (取绝对值的平均)
# g_shap = np.abs(shap_values).mean(axis=0)

# # 生成排名表并计算相对重要性
# shap_df = pd.DataFrame({
#     'Feature': X_train_scaled.columns,
#     'G_SHAP_Value': g_shap
# }).sort_values('G_SHAP_Value', ascending=False)

# shap_df['Relative'] = shap_df['G_SHAP_Value'] / shap_df['G_SHAP_Value'].sum()

# print("G-SHAP values with relative importance:")
# print(shap_df[['Feature', 'G_SHAP_Value', 'Relative']])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import resample
import shap
import pandas as pd
from sklearn.utils import resample

base_features = X_train.columns.tolist()
experiment_data_refined = {}

for w, data in experiment_data.items():
    X_tr_3d = data['X_train']
    N, L, D = X_tr_3d.shape
    
    # 步骤 2 & 3: 展平为2D矩阵，训练随机森林并计算SHAP
    X_tr_flat = X_tr_3d.reshape(N, L * D)
    rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf.fit(X_tr_flat, data['y_train'])

    # [优化]: 对 SHAP 解释集进行降采样 (最多取 1000 条)
    sample_size = min(1000, N)
    X_shap_sample = resample(X_tr_flat, n_samples=sample_size, random_state=42, replace=False)

    shap_values = shap.TreeExplainer(rf).shap_values(X_shap_sample, check_additivity=False)
    
    # 步骤 4: 时间维度的SHAP聚合 (先求每个展开特征的绝对均值，再按时间步L聚合求均值)
    mean_abs_shap = np.abs(shap_values).mean(axis=0)
    g_shap_base = mean_abs_shap.reshape(L, D).mean(axis=0)
    
    # 生成基础变量的全局重要度排名
    shap_df = pd.DataFrame({'Feature': base_features, 'G_SHAP_Value': g_shap_base})
    shap_df = shap_df.sort_values('G_SHAP_Value', ascending=False)
    shap_df['Relative'] = shap_df['G_SHAP_Value'] / shap_df['G_SHAP_Value'].sum()
    
    print(f"\nWindow Size (L={w}) - Aggregated G-SHAP values:")
    print(shap_df[['Feature', 'G_SHAP_Value', 'Relative']])
    
    # 步骤 5: 变量剔除与重构 3D 矩阵 (保留累积重要度Top 85%的特征)
    cutoff = np.argmax(shap_df['Relative'].cumsum().values >= 0.85) + 1
    selected_idx = [base_features.index(f) for f in shap_df.head(cutoff)['Feature']]
    
    experiment_data_refined[w] = {
        'X_train': data['X_train'][:, :, selected_idx], 'y_train': data['y_train'],
        'X_val': data['X_val'][:, :, selected_idx], 'y_val': data['y_val'],
        'X_test': data['X_test'][:, :, selected_idx], 'y_test': data['y_test']
    }


Window Size (L=5) - Aggregated G-SHAP values:
   Feature  G_SHAP_Value  Relative
5   US10YT      0.000218  0.124673
0     Gold      0.000176  0.101034
4     USDX      0.000158  0.090587
9      GPR      0.000151  0.086401
1      WTI      0.000149  0.085627
2   Silver      0.000145  0.083343
3     DJIA      0.000135  0.077148
8      VIX      0.000125  0.071677
12     BBW      0.000112  0.064055
7   T10YIE      0.000087  0.049835
13    MACD      0.000079  0.045052
10     CPI      0.000073  0.042052
6     EFFR      0.000071  0.040405
11     RSI      0.000067  0.038109

Window Size (L=21) - Aggregated G-SHAP values:
   Feature  G_SHAP_Value  Relative
0     Gold      0.000058  0.111345
5   US10YT      0.000057  0.110217
4     USDX      0.000048  0.092574
12     BBW      0.000047  0.090257
2   Silver      0.000045  0.087619
3     DJIA      0.000041  0.078602
9      GPR      0.000040  0.077687
8      VIX      0.000038  0.073980
1      WTI      0.000038  0.073245
6     EFFR      0.000025  0.04

## 最优特征子集

In [42]:
# 依据指定的保留特征，直接构造最优特征子集
selected_features = ['Gold', 'US10YT', 'USDX', 'GPR', 'WTI', 'Silver', 'DJIA', 'VIX', 'BBW']
selected_idx = [base_features.index(f) for f in selected_features]

experiment_data_refined = {}

for w, data in experiment_data.items():
    experiment_data_refined[w] = {
        'X_train': data['X_train'][:, :, selected_idx], 'y_train': data['y_train'],
        'X_val': data['X_val'][:, :, selected_idx], 'y_val': data['y_val'],
        'X_test': data['X_test'][:, :, selected_idx], 'y_test': data['y_test']
    }

In [43]:
import pickle

# 将筛选后的最优特征数据集保存到本地文件
with open('../data/processed/experiment_data_refined.pkl', 'wb') as f:
    pickle.dump(experiment_data_refined, f)
    
print("The filtered feature data has been successfully saved as experiment_data_refined.pkl")

The filtered feature data has been successfully saved as experiment_data_refined.pkl
